In [47]:
import hazelcast
from threading import Thread, Lock

In [40]:
client = hazelcast.HazelcastClient(
    cluster_name="hello-world",
)

my_map = client.get_map("my-distributed-map")

Non-blocking counter

In [119]:
my_map.put(key="counter", value=0)

In [121]:
def increment_counter(increment_value=10000):
    for _ in range(increment_value):
        counter_value = my_map.get("counter").result()
        my_map.put(key="counter", value=counter_value + 1)

In [122]:
%%time
threads = []
for i in range(10):
    thread = Thread(target=increment_counter)
    threads.append(thread)
    thread.start()

for thr in threads:
    thr.join()

CPU times: user 10.4 s, sys: 768 ms, total: 11.2 s
Wall time: 10.9 s


In [123]:
print(f"Finished! Result = {my_map.get('counter').result()}")

Finished! Result = 10001


Pessimistic-locking counter

In [53]:
my_map.put(key="counter", value=0)

In [54]:
counter_lock = Lock()

In [55]:
def increment_counter_locking(increment_value=10000):
    for _ in range(increment_value):
        with counter_lock:
            counter_value = my_map.get("counter").result()
            my_map.put(key="counter", value=counter_value + 1)

In [56]:
%%time
threads = []
for i in range(10):
    thread = Thread(target=increment_counter_locking)
    threads.append(thread)
    thread.start()

for thr in threads:
    thr.join()

CPU times: user 1min 2s, sys: 10.5 s, total: 1min 12s
Wall time: 6min 6s


In [57]:
print(f"Finished! Result = {my_map.get('counter').result()}")

Finished! Result = 100000


Optimistic-locking counter

In [67]:
my_map.put(key="counter", value=0)

In [68]:
def increment_counter_replacing(increment_value=10000):
    for _ in range(increment_value):
        with counter_lock:
            old_counter_value = my_map.get("counter").result()
            new_counter_value = old_counter_value + 1
            my_map.replace_if_same("counter",
                                   old_counter_value, new_counter_value)

In [69]:
%%time
threads = []
for i in range(10):
    thread = Thread(target=increment_counter_replacing)
    threads.append(thread)
    thread.start()

for thr in threads:
    thr.join()

CPU times: user 1min 3s, sys: 10.4 s, total: 1min 14s
Wall time: 6min 16s


In [70]:
print(f"Finished! Result = {my_map.get('counter').result()}")

Finished! Result = 100000


IAtomicLong with CP Sysbsystem counter

In [115]:
my_atomic_long = client.cp_subsystem.get_atomic_long("my-counter")
my_atomic_long.set(0)

In [116]:
def increment_counter_atomic_long(increment_value=10000):
    for _ in range(increment_value):
        my_atomic_long.get_and_increment()

In [117]:
%%time
threads = []
for i in range(10):
    thread = Thread(target=increment_counter_atomic_long)
    threads.append(thread)
    thread.start()

for thr in threads:
    thr.join()

CPU times: user 1.87 s, sys: 60.8 ms, total: 1.93 s
Wall time: 1.9 s


In [118]:
print(f"Finished! Result = {my_atomic_long.get().result()}")

Finished! Result = 100000
